In [86]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Data Exploration
I want to scrape the Purdue butcher block website to see when they update it, scrape the results, and provide an easily navigatable interface (and tell me if they have chuck eyes)

In [91]:
url = 'https://ag.purdue.edu/ansc/ButcherBlock/Pages/Meatlist.aspx'
html = requests.get(url,verify=False).text

/home/will/Projects/butcher_block/env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ag.purdue.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


All of the values I want to parse are located in \<tr\> elements, so I can use BeautifulSoup to parse out the table rows

In [92]:

soup = BeautifulSoup(html,'html.parser')
rows = soup.find_all('tr')

Thhe values are nested in \<td\> elements inside of the table rows, in an attribute called data-sheets-value. I can extract each of these into a list

In [93]:
import json
for row in rows:
    curr = []
    for td in row:
        if 'data-sheets-value' in td.attrs:
            #print(td['data-sheets-value'])
            val = json.loads(td['data-sheets-value'])
            if '2' in val:
                curr.append(val['2'])
            else:
                curr.append(val['3'])

It gets a little complicated, but basically the header is Category | Avg Weight | $/lb
The rows arent always lined up header to header, so I need to dynamically figure out when the headers per row are starting/stopping

In [94]:
import json
h1 = None
h2 = None
d1 = {}
run1 = []
run2 = []
for i,row in enumerate(rows[3:]):
    tds = [td.string for td in row if 'data-sheets-value' in td.attrs]
    if i % 5 == 0:
        print(tds)
    #print(run1)
    vals = [td for td in tds]
    vals = [v for v in vals if v != ' '] 
    #print(f'current row vals:{vals}')
    col1,col2 = vals[:3],vals[3:]
    #print(f'col1:{col1}\ncol2:{col2}')
    #print(f'run1:{run1}\nrun2:{run2}')
    if 'Average Weight' in col1:
        if h1 == None:
            h1 = col1[0]
        else:
            d1[h1] = run1
            h1 = col1[0]
            run1 = []
    elif col1!=[] and col2 != []:
        run1.append(col1)
    elif col2 == []:
        run2.append(col1)
    if 'Average Weight' in col2:
        if h2 == None:
            h2 = col2[0]
        else:
            d1[h2] = run2
            h2 = col2[0]
            run2 = []
    elif col2 != None:
        run2.append(col2)




['Beef Steaks', 'Average Weight', 'Price Per Pound', 'Pork Chops, Steaks & Ribs', 'Average Weight', 'Price Per Pound']
['Flat Iron', '1', '$8.99', 'Pork Roasts', 'Average Weight', 'Price Per Pound']
['Beef Roasts', 'Average Weight', 'Price Per Pound']
['Rump Roast ', '3-5', '$5.99', 'Beef Patties', '6', '$6.99']
['Beef Stew Meat', '1-2', '$5.99', 'Southern Brand Pork Sausage', '1-2', '$3.99']
['Boneless Breast', '1', '$5.99']
['Mozzarella Brats', '1.75', '$3.99', 'Whole Chicken', '3', '$1.99']
['Wings', '2', '$1.99', 'Buffalo Style Turkey Wings', '2', '$2.99']
['Smoked and Cured', 'Average Weight', 'Price Per Pound']
['Pork Summer Sausage', '1', '$4.99', 'Leg Roast', '7', '$7.99']
['Boiler Tracks Family Size', '60 oz', '$13.50', 'Loin Roast', '3', '$12.99']
[]


I got it, besides some additional empty lists that were added to the dictionary. I can now create a pandas df 

In [95]:
columns = ['item','avg weight/size','price/lb|unit']
df_dict = {k : pd.DataFrame(v,columns=columns) for k,v in d1.items()}

df = (pd.concat(df_dict)
        .reset_index(level=1,drop=True)
        .rename_axis('category')
        .reset_index()
        .dropna()
)

In [96]:
df.head()

,category,item,avg weight/size,price/lb|unit
0,"Pork Chops, Steaks & Ribs",Fresh Ham Steak,1,$1.89
1,"Pork Chops, Steaks & Ribs",Boneless America's Chops,1.5,$3.99
2,"Pork Chops, Steaks & Ribs",Blade Steaks,1.5,$1.49
4,Beef Steaks,Top Round,2,$4.99
5,Beef Steaks,Eye of Round,1,$5.99


Got it into a dataframe, now I want to be able to select category, and sort by weight/price.

I also will identify sale items, and put those on the main page

In [97]:
df = df.applymap(str)

In [98]:

def weight_range_to_avg(s):
    if '-' in s:
        vals = s.split('-')
        return sum([int(val) for val in vals])/len(vals)
    return float(s)
def dollar_string_to_float(s):
    if '$' in s:
        return float(s[1:])
    return float(s)

df['avg weight/size'] = df['avg weight/size'].map(weight_range_to_avg)
df['price/lb|unit'] = df['price/lb|unit'].map(dollar_string_to_float)



In [99]:
df['sale'] = df.item.str.contains('\*')

In [100]:
df.head()

,category,item,avg weight/size,price/lb|unit,sale
0,"Pork Chops, Steaks & Ribs",Fresh Ham Steak,1.0,1.89,False
1,"Pork Chops, Steaks & Ribs",Boneless America's Chops,1.5,3.99,False
2,"Pork Chops, Steaks & Ribs",Blade Steaks,1.5,1.49,False
4,Beef Steaks,Top Round,2.0,4.99,False
5,Beef Steaks,Eye of Round,1.0,5.99,False


In [85]:
df.to_csv('butcher_block02-17.csv')